In [3]:
import json as js
import pandas
from collections import defaultdict

def OpenData():
    with open("E://1数据挖掘/全部数据/train_data_complete.json",'r', encoding='UTF-8') as load_f:
        train_data_sample = js.load(load_f)
        #print(train_data_sample[4])
        
    with open("E://1数据挖掘/全部数据/test_data_sample.json",'r', encoding='UTF-8') as load_f:
        test_data_sample = js.load(load_f)
        #print(test_data_sample[4])
    return train_data_sample,test_data_sample

In [4]:
train_data_sample,test_data_sample = OpenData()

In [6]:
train_data_sample[0]

{'item_id': 100001,
 'passages': [{'content': '高速占用应急车道行驶扣几分', 'label': 0, 'passage_id': 10000101},
  {'content': '高速公路的应急车道是“生命通道”,很多朋友产生疑惑,换驾驶员、困了想临时休息、“方便”、打电话、为',
   'label': 0,
   'passage_id': 10000102},
  {'content': '6分。', 'label': 1, 'passage_id': 10000103},
  {'content': '违法占用应急车道扣几分? 核心内容:车辆在高速公路上行驶时候,违法占用应急车道要不要扣分?要扣分。那么 要扣几分?按照法律规定,违法占用应急车道要记6分,罚款200元。法律快车小编为您提 ...',
   'label': 1,
   'passage_id': 10000104},
  {'content': '高速上停车给孩子换尿片,可以占用应急车道吗? 依据新交规,机动车在城市快速路、高速公路上遇交通拥堵时,非紧急情况下在应急车道、路肩上停车的,均罚款200元,记6分,...',
   'label': 1,
   'passage_id': 10000105},
  {'content': '占用高速应急车道开车在《中华人民共和国道路交通安全法》中并没有详细的规定,但法规第九十条规定:机动车驾驶人违反道路交通安全法律、法规关于道路通行规定的,处警告或者二十元以上二百元以下罚款。 ...',
   'label': 0,
   'passage_id': 10000106},
  {'content': '5分', 'label': 0, 'passage_id': 10000107},
  {'content': '2017高速公路占用应急车道扣几分,应急车道顾名思义就是给专门处理紧急事故的车走的,一般我们在高速上行驶是不允许走应急车道的,那2017高速公路占用应急车道扣几分呢?...',
   'label': 0,
   'passage_id': 10000108},
  {'content': '依据新交规,机动车在城市快速路、高速公路上遇交通拥堵时,非紧急情况下在应急车道、路肩上停车的,均罚款

In [4]:
#提取出问题答案的文字

A,L,item_id,Q = [],[],[],[]                 #储存为 问题 答案 标签
#print(len(load_dict))
for i in range(0,27000):
    for j in range(0,len(train_data_sample[i]['passages'])):
        A.append(train_data_sample[i]['passages'][j]['content'])                     #答案
        #print(A)
        L.append(train_data_sample[i]['passages'][j]['label'])                       #标签
        item_id.append(train_data_sample[i]['item_id'])        #id
        Q.append(train_data_sample[i]['question'])
      

    

A2,L2,item_id2,Q2 = [],[],[],[]                 #储存为 问题 答案 标签
#print(len(load_dict))
for i in range(27000,len(train_data_sample)):
    for j in range(0,len(train_data_sample[i]['passages'])):
        A2.append(train_data_sample[i]['passages'][j]['content'])                     #答案
        #print(A)
        L2.append(train_data_sample[i]['passages'][j]['label'])                       #标签
        item_id2.append(train_data_sample[i]['item_id'])        #id
        Q2.append(train_data_sample[i]['question'])

In [13]:
c0,c1 = 0,0
for i in L:
    if i==1:
        c0+=1
    else:
        c1+=1
for i in L2:
    if i==1:
        c0+=1
    else:
        c1+=1
print(c0,c1)

127328 349691


In [12]:
print(len(A)+len(A2))

477019


In [5]:
def addstopwords():
    #询问数字
    jieba.add_word('几')
    jieba.del_word('几年')
    jieba.del_word('几月')
    jieba.del_word('几日')
    jieba.del_word('几个')
    jieba.add_word('多少')
    #询问时间
    jieba.add_word('什么时候')
    jieba.add_word('何时')
    
    #询问原因
    jieba.add_word('为什么')
    jieba.add_word('为何')
    jieba.add_word('什么原因')
    #询问地方
    jieba.add_word('哪里')
    jieba.add_word('哪个')
    jieba.add_word('那里')
    #询问动作
    jieba.add_word('怎么')
    jieba.add_word('如何')
    jieba.add_word('怎样')
    #询问人
    jieba.add_word('谁')

In [6]:
import jieba
addstopwords()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.065 seconds.
Prefix dict has been built succesfully.


In [7]:
list(jieba.cut(Q[0].strip(),cut_all=False))

['高速', '占用', '应急', '车道', '扣', '多少', '分']

In [9]:
list(jieba.cut('天空为什么是蓝色的'.strip(),cut_all=False))

['天空', '为什么', '是', '蓝色', '的']

In [8]:
'_'.join(jieba.cut('世界上有多少个国家?'.strip(),cut_all=False))

'世界_上_有_多少_个_国家_?'

In [9]:
import time
print (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
max_len = 0
sum_len = 0
for i in range(0,len(A)):
    c_len = len(list(jieba.cut(A[i].strip(),cut_all=False)))
    if c_len>max_len:
        max_len = c_len
        #print('更新长度')
    else:
        pass
    sum_len+=c_len
mean_len = sum_len/len(A)        
print('最大长度为',max_len)
print('平均长度为',mean_len)
print (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

2018-04-09 17:23:26
最大长度为 2406
平均长度为 60.027042003768145
2018-04-09 17:24:14


In [1]:
def padding(line):
    _len = 100
    if len(line)<_len:
        for pa in range(_len-len(line)):
            line.append('<a>')
    else:
        line = line[0:100]
        pass
    return line
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  

# 对句子进行分词  
def rm_tokens(sentence): 
    stopwords = stopwordslist('stop_words.txt')  
    output = []
    for word in sentence:  
        if word not in stopwords:  
            if word != '\t':  
                output.append(word)
    return output

In [2]:
stopwords = stopwordslist('stop_words.txt')
print(''.join(stopwords))

﻿一二三四五六七八九十零请？“”；：，。/《》?一个"';:?.,/()-_--*/-+你我他它她这请问你好您好世界上世界第_________><的地得！＿、


In [14]:
print(rm_tokens(jieba.cut('世界上•有 几个国家',cut_all=False)))
'_'.join(padding(list(rm_tokens(jieba.cut('世界上       有•几个国家'.strip().replace(' ',''),cut_all=False)))))

['上', '•', '有', ' ', '几', '个', '国家']


'上_有_•_几_个_国家_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>_<a>'

In [15]:
print(len(A),len(Q),len(A2),len(Q2))

429844 429844 47175 47175


In [10]:
len(train_data_sample)

30000

In [16]:
import time
train_list = []
test_list = []
print (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
#for i in range(0,5):
for i in range(0,len(A)):
    tq = '_'.join(padding(list(rm_tokens(jieba.cut(Q[i].strip().replace(' ',''),cut_all=False)))))
    ta = '_'.join(padding(list(rm_tokens(jieba.cut(A[i].strip().replace(' ',''),cut_all=False)))))
    tl = 'qid:'+str(item_id[i])
    train_list.append(' '.join([str(L[i]),tl,tq,ta]))
print (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
#for i in range(0,5):
for i in range(0,len(A2)):
    tq2 = '_'.join(padding(list(rm_tokens(jieba.cut(Q2[i].strip().replace(' ',''),cut_all=False)))))
    ta2 = '_'.join(padding(list(rm_tokens(jieba.cut(A2[i].strip().replace(' ',''),cut_all=False)))))
    tl2 = 'qid:'+str(item_id2[i])
    test_list.append(' '.join([str(L[i]),tl2,tq2,ta2]))
print (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

2018-04-12 15:25:16
2018-04-12 15:33:32
2018-04-12 15:34:26


In [17]:
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

ntrain_list = '\n'.join(train_list)

print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

ntest_list = '\n'.join(test_list)

print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

2018-04-12 15:34:33
2018-04-12 15:34:34
2018-04-12 15:34:34


In [18]:
import re
f = open("D://1数据挖掘/C_pre_data/train", "w", encoding='utf-8')
f.write(ntrain_list)
f.close()

f = open("D://1数据挖掘/C_pre_data/test", "w", encoding='utf-8')
f.write(ntest_list)
f.close()

print('训练/测试数据写入成功')

训练/测试数据写入成功
